## Perform and save harmonic fit for Ichthys data

Fitting the records of currents from P1, CPF, Titan, and Fblock

Author: William Edge

Created: 04/10/23

In [10]:
import os
import numpy as np
import xarray as xr
from ichthys_harmonics.utils import pop_timevars, murphy_ss
from ichthys_harmonics.harmonic_funcs import pca_rotate_data
from ichthys_harmonics.harmonic_lstsq import harmonic_fit, least_squares_prediction, print_comb_skillscores

#### Load mooring data

In [11]:
### Select mooring
# moor = 'IchthysP1'
# moor = 'Fblock'
# moor = 'Ichthys_CPF'
moor = 'titanichthys'

# Load data
ds = xr.open_dataset(os.path.join('measured_data', moor + '_nearbed_currents_10minutes.nc'))

In [12]:
# Barotropic rotations
nanx = ~np.isnan(ds['v_depavg'].values)
U_rot, V_rot, theta = pca_rotate_data(ds['u_depavg'].values[nanx], ds['v_depavg'].values[nanx])

# Barotropic North-South tide: run and save
ha_ls_BTNS = harmonic_fit(ds['time'].values[nanx], V_rot, ds_attrs=ds.attrs)
ha_ls_BTNS.attrs['pca_angle'] = theta
ha_ls_BTNS = pop_timevars(ha_ls_BTNS)
fname = os.path.join('fits', moor + '_BTNS_harmonic_fit.nc')
ha_ls_BTNS.to_netcdf(fname)

# Barotropic East-West tide: run and save
ha_ls_BTEW = harmonic_fit(ds['time'].values[nanx], U_rot, ds_attrs=ds.attrs)
ha_ls_BTEW.attrs['pca_angle'] = theta
ha_ls_BTEW = pop_timevars(ha_ls_BTEW)
fname = os.path.join('fits', moor + '_BTEW_harmonic_fit.nc')
ha_ls_BTEW.to_netcdf(fname)


# Baroclinic rotations
nanx = ~np.isnan(ds['v_meas'].values)
U_rot, V_rot, theta = pca_rotate_data((ds['u_meas'].values - ds['u_depavg'].values)[nanx],\
                                      (ds['v_meas'].values - ds['v_depavg'].values)[nanx])

# Baroclinic North-South tide: run and save
ha_ls_ITNS = harmonic_fit(ds['time'].values[nanx], V_rot, ds_attrs=ds.attrs)
ha_ls_ITNS.attrs['pca_angle'] = theta
ha_ls_ITNS = pop_timevars(ha_ls_ITNS)
fname = os.path.join('fits', moor + '_ITNS_harmonic_fit.nc')
ha_ls_ITNS.to_netcdf(fname)

# Baroclinic East-West tide: run and save
ha_ls_ITEW = harmonic_fit(ds['time'].values[nanx], U_rot, ds_attrs=ds.attrs)
ha_ls_ITEW.attrs['pca_angle'] = theta
ha_ls_ITEW = pop_timevars(ha_ls_ITEW)
fname = os.path.join('fits', moor + '_ITEW_harmonic_fit.nc')
ha_ls_ITEW.to_netcdf(fname)

# Print the total skill score
print_comb_skillscores(ds, ha_ls_BTNS, ha_ls_BTEW, ha_ls_ITNS, ha_ls_ITEW)

Skill score: 83.43%
Skill score: 98.13%
Skill score: 61.95%
Skill score: 19.28%
North-South combined score: 84.55%
East-West combined score: 74.59%
Total speed score: 62.57%
